In [1]:
import pandas as pd

df = pd.read_csv("Files/mer11_ABCD_chipatlas_f05_F05_e_Feb2020.tsv.gz", sep='\t')


In [2]:
df = df[df['Group'] == 'TFs and others']

In [3]:
import numpy as np
df['score'] = -np.log10(df['FisherFDR'])
df['score'].fillna(320, inplace=True)
df['score'].replace(np.inf, 320, inplace=True)
df['score'].replace(-0.0, 0, inplace=True)

/home/ndlo201/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
#MER11D not part of the same family as MER11A/B/C so discard
df = df[df['MERFam'] != 'MER11D']

#discard hits with tags
df = df[df['TF'] != 'GFP']
df = df[df['TF'] != 'Epitope tags']

In [5]:
df_hits = df.loc[df.groupby(['TF'])['FisherFDR'].idxmin()].drop(['Description', 'MeanPeakLength', 'StdPeakLength', 'RP', 'BP'], axis=1)

#keep only hits with pvalue -100 or less
df_hits = df_hits.loc[(df_hits['FisherFDR'] < 10 ** -100)]
df_hits = df.loc[df['ID'].isin(df_hits['ID'])]

In [6]:
import pandas as pd
import seaborn as sns
from bokeh.plotting import show, figure
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, BasicTicker, PrintfTickFormatter, HoverTool
from bokeh.palettes import Viridis256
from bokeh.transform import transform
from bokeh.io import output_notebook
from bokeh.io import output_notebook, output_file, export_png, export_svgs
import numpy as np

output_notebook()

d = {'category': df_hits['MERFam'],
     'factor': df_hits['TF'],
     'score': df_hits['score'],
     'enrichment': df_hits['OR'].clip(0,40)}


df_plot = pd.DataFrame(data = d)
df_plot.sort_values(['category', 'factor'], inplace=True)
print(df_plot[df_plot['factor'] == 'EP300'])
source = ColumnDataSource(df_plot)
p = figure(x_range = df_plot['category'].unique(), y_range = df_plot['factor'].unique()[::-1], plot_width=400, plot_height=2000)

color_mapper = LinearColorMapper(palette=["#FFFFFF"] + sns.color_palette("viridis", 256).as_hex(), low = 0, high = 320)
color_bar = ColorBar(color_mapper = color_mapper, location = (0, 0), ticker = BasicTicker())
p.add_layout(color_bar, 'right')
p.scatter(x='category', y='factor', size='enrichment', fill_color=transform('score', color_mapper), source=source, line_color='#333333')
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xgrid.visible = False
p.ygrid.visible = False
p.add_tools(HoverTool(tooltips = [('score', '@score'), ('enrichment', '@enrichment')]))

filename = "Chip-atlas"
p.output_backend = "svg"
# export_svgs(p, filename=filename + ".svg")
# export_png(p, filename=filename + ".png")

show(p)

Loading BokehJS ...

      category factor       score  enrichment
5910    MER11A  EP300  320.000000        40.0
47506   MER11B  EP300  320.000000        40.0
89102   MER11C  EP300  315.603869        40.0
